In [24]:
import requests
import pandas as pd
import bs4

In [25]:
url = "https://www.counterstats.net/"

In [26]:
req = requests.get(url)
soup = bs4.BeautifulSoup(req.text)

In [27]:
divs = soup.find_all('span', class_ = 'name')
names = [div.text.lower().replace(" ", "-").replace("'", "") for div in divs]

In [28]:

df = pd.read_csv('complete_cleaned.csv')

def format_champion(champ):
    if pd.isna(champ):  
        return None  
    return champ.lower().replace(" ", "-").replace("'", "")

# Apply function to each element in the selected columns
cols = ['ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5']
df[cols] = df[cols].applymap(format_champion)

# Display head of the DataFrame
df.head()


C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_7536\1536760989.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(format_champion)


,Unnamed: 0,year,gameid,league,teamname,side,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,result
0,0,2017,1506-1540,LPL,I May,Blue,syndra,malzahar,ashe,karma,poppy,maokai,khazix,cassiopeia,varus,tahm-kench,1
1,1,2017,1506-1540,LPL,Royal Never Give Up,Red,camille,rengar,zyra,elise,reksai,kled,lee-sin,ryze,caitlyn,nautilus,0
2,2,2017,1506-1541,LPL,I May,Blue,syndra,malzahar,ashe,reksai,khazix,maokai,lee-sin,corki,caitlyn,thresh,1
3,3,2017,1506-1541,LPL,Royal Never Give Up,Red,rengar,camille,varus,cassiopeia,orianna,trundle,rumble,ryze,jhin,zyra,0
4,4,2017,1507-1544,LPL,Invictus Gaming,Blue,jayce,elise,malzahar,khazix,lee-sin,singed,rengar,leblanc,varus,tahm-kench,1


In [29]:
def wr(names, df, columns_to_check):    

    wr_dict = {} 

    for name in names:  
        games_with_name = df[df[columns_to_check].isin([name]).any(axis=1)]
        num_wins = games_with_name[games_with_name['result'] == 1].shape[0]
        
        total_games = games_with_name.shape[0]

        win_rate = num_wins / total_games if total_games > 0 else 0
        if name == 'nunu-&-willump': 
            wr_dict['nunu-willump'] = win_rate
            continue
        if name == 'dr.-mundo': 
            wr_dict['dr-mundo'] = win_rate
            continue

        
        wr_dict[name] = win_rate
        
    return wr_dict

champ_wr = wr(names, df, ['pick1', 'pick2', 'pick3', 'pick4', 'pick5'])



In [30]:
def download_page(champ):
    url = f'https://www.counterstats.net/league-of-legends/{champ}'
    request = requests.get(url)
    return bs4.BeautifulSoup(request.text, 'html.parser')


def parse(names):
    data = []
    
    for name in names:
        print(name)

        name = name.replace("nunu-&-willump", "nunu-willump").replace("dr.-mundo", "dr-mundo")

        soup = download_page(name)  
        
        counters_champ_div = soup.find_all("div", class_='champ-box')[0]
        champ_counter_div = soup.find_all("div", class_='champ-box')[1]


        champ_counters = []
        counters_champ = []

        for i in range(5): 

            divs = counters_champ_div.find_all("div", class_='win-wrap')
            
            div = divs[i]
            
            champion_tag = div.find("span", class_="champion").text.strip()

            win_tag = div.find("span", class_="win").find("span", class_="b").text
            win_percent = 100-int(win_tag.replace("%", ""))

            counters_champ.append((champion_tag, win_percent))

        for i in range(5): 
            divs = champ_counter_div.find_all("div", class_='win-wrap')

            div = divs[i]
            
            champion_tag = div.find("span", class_="champion").text

            win_tag = div.find("span", class_="win").find("span", class_="b").text
            win_percent = 100-int(win_tag.replace("%", ""))

            champ_counters.append((champion_tag, win_percent))
        
        for i in range(5): 
            data.append([name, counters_champ[i], champ_counters[i]])
            df = pd.DataFrame(data, columns=["Champion", "counters champ", "champ counters"])
            df.set_index("Champion", inplace=True) 


    return df


result = parse(names)
result['champ_wr'] = wr(names, df, ['pick1', 'pick2', 'pick3', 'pick4', 'pick5'])

darius
nilah
taric
jinx
nami
vayne
nocturne
warwick
braum
kayle
yorick
janna
ornn
anivia
cassiopeia
quinn
urgot
annie
kennen
kled
kogmaw
olaf
sona
lulu
karma
thresh
diana


KeyboardInterrupt: 

In [ ]:
result.to_csv('champs_counters_wr.csv')

In [ ]:
import numpy as np 
result[result['champ_wr'].isna()]

,counters champ,champ counters,champ_wr
Champion,,,


In [ ]:
(champ_wr['maokai'] + champ_wr['khazix'] + champ_wr['cassiopeia'] + champ_wr['varus'] + champ_wr['tahm-kench'])/5

0.5086664513376833